# RePsychLing Masson, Rabe, &amp; Kliegl, 2017) with Julia
### Reinhold Kliegl
### 2020-02-13
# Setup

Packages we (might) use.

In [1]:
#cd(joinpath(homedir(),"Google Drive/ZiF_CG_WS2/MRK17_Exp1/"))
#cd("/Users/reinholdkliegl/Google Drive/ZiF_CG_WS2/MRK17_Exp1/")

using CSV, DataFrames, DataFramesMeta, MixedModels, RCall 
using StatsBase, StatsModels, BenchmarkTools, Weave

# needed for LRT
using Printf: @sprintf
using Distributions: Chisq, ccdf
include("lrtest.jl")

┌ Info: Precompiling MixedModels [ff71e718-51f3-5ec2-a782-8ffcbfa3c316]
└ @ Base loading.jl:1273


In [3]:
R"dat_r = readRDS('MRK17_Exp1.rds')";

dat = rcopy(R"dat_r")

dat = @linq dat |>
       transform(F = levels!(:F, ["HF", "LF"]),
                 P = levels!(:P, ["rel", "unr"]),
                 Q = levels!(:Q, ["clr", "deg"]),
                lQ = levels!(:lQ, ["clr", "deg"]),
                lT = levels!(:lT, ["WD", "NW"]))

cellmeans = by(dat, [:F, :P, :Q, :lQ, :lT], 
            meanRT = :rt => mean, sdRT = :rt => std, n = :rt => length,
            semean = :rt => x -> std(x)/sqrt(length(x)))

,F,P,Q,lQ,lT,meanRT,sdRT
,Categorical…,Categorical…,Categorical…,Categorical…,Categorical…,Float64,Float64
1,HF,rel,clr,clr,WD,613.094,192.13
2,HF,rel,clr,clr,NW,635.319,205.19
3,HF,rel,clr,deg,WD,620.569,173.764
4,HF,rel,clr,deg,NW,615.647,176.826
5,HF,rel,deg,clr,WD,667.685,231.145
6,HF,rel,deg,clr,NW,645.493,179.962
7,HF,rel,deg,deg,WD,637.793,177.496
8,HF,rel,deg,deg,NW,659.266,192.637
9,HF,unr,clr,clr,WD,620.838,192.689


# Reading data

We read the data preprocessed with R and saved as RDS file (see `DataPrep.Rmd` for details).

# Complex LMM

This is *not* the maximal factorial LMM because we do not include interaction 
terms and associated correlation parameters in the RE structure.

## Model fit

In [4]:
const HC = HelmertCoding();
const contrasts = Dict(:F => HC, :P => HC, :Q => HC, :lQ => HC, :lT => HC);

m1form = @formula (-1000/rt) ~ 1+F*P*Q*lQ*lT +
                              (1+F+P+Q+lQ+lT | Subj) +
                              (1+P+Q+lQ+lT | Item);
cmplxLMM = @btime fit(MixedModel, m1form, dat, contrasts=contrasts);

  12.373 s (422033 allocations: 144.88 MiB)


## VCs and CPs

We don't look at fixed effects before model selection.

In [5]:
cmplxLMM.λ[1]
cmplxLMM.λ[2]
cmplxLMM.rePCA

(Item = [0.877285952024819, 0.9515421973498676, 0.9915019239085304, 1.0, 1.0], Subj = [0.9421686601104765, 0.9774140947628489, 0.9961956271378415, 0.9984374010364477, 1.0, 1.0])

Variance-covariance matrix of random-effect structure suggests overparameterization
for both subject-related and item-related components.


# Zero-correlation parameter LMM (factors)

## Model fit

We take out correlation parameters.

In [6]:
m2form = @formula (-1000/rt) ~ 1 + F*P*Q*lQ*lT +
                               zerocorr(1+F+P+Q+lQ+lT | Subj) +
                               zerocorr(1+P+Q+lQ+lT | Item);

zcpLMM = @btime fit(LinearMixedModel, m2form, dat, contrasts=contrasts);

  2.928 s (245260 allocations: 137.40 MiB)


## VCs and CPs

In [7]:
zcpLMM.λ[1]
zcpLMM.λ[2]
zcpLMM.rePCA

(Item = [0.9008661656020007, 0.9424502421207154, 0.9751854754242245, 0.9955462479417729, 1.0], Subj = [0.9367061661526143, 0.9691670136862988, 0.9928321057825136, 0.9964771278392464, 0.999523592146044, 1.0])

Looks ok, but the last PCs are very tiny. Might be a good idea to prune the LMM. 

# Zero-correlation parameter LMM (indicators)

An alternative solution is to extract the indicators of contrasts from the design matrix.
Sometimes RE structures are more conviently specified with indicator variables (i.e., 
@ level of contrasts) than the factors.

In [10]:
mm = Int.(zcpLMM.X)

dat = @linq dat |>
       transform(f = mm[:, 2],
                 p = mm[:, 3],
                 q = mm[:, 4],
                lq = mm[:, 5],
                lt = mm[:, 6]);
dat[1:10, 10:14]

,f,p,q,lq,lt
,Int64,Int64,Int64,Int64,Int64
1,1,1,1,1,1
2,-1,1,-1,1,-1
3,-1,-1,-1,1,1
4,-1,-1,1,1,1
5,-1,1,1,1,-1
6,-1,-1,-1,1,-1
7,-1,-1,1,1,1
8,1,1,1,-1,1
9,1,1,-1,1,-1


We take out correlation parameters.

In [11]:
m2form_b = @formula (-1000/rt) ~ 1 + f*p*q*lq*lt +
 (1 | Subj) + (0+f | Subj) + (0+p | Subj) + (0+q | Subj) + (0+lq | Subj) + (0+lt | Subj) +
 (1 | Item) +                (0+p | Item) + (0+q | Item) + (0+lq | Item) + (0+lt | Item);

zcpLMM_b = @btime fit(LinearMixedModel, m2form_b, dat, contrasts=contrasts)

  2.777 s (207077 allocations: 134.36 MiB)


In [13]:
const mods = [cmplxLMM, zcpLMM, zcpLMM_b];
gof_summary = DataFrame(dof=dof.(mods), deviance=deviance.(mods),
              AIC = aic.(mods), AICc = aicc.(mods), BIC = bic.(mods))

,dof,deviance,AIC,AICc,BIC
,Int64,Float64,Float64,Float64,Float64
1,69,7147.55,7285.55,7286.14,7817.24
2,44,7188.49,7276.49,7276.73,7615.54
3,44,7188.49,7276.49,7276.73,7615.54


In [14]:
lrtest(cmplxLMM, zcpLMM)

┌ Warning: Could not check whether models are nested as model type LinearMixedModel does not implement isnested: results may not be meaningful
└ @ Main /Users/reinholdkliegl/Google Drive/ZiF_CG_WS2/MRK17_Exp1/lrtest.jl:91


Likelihood-ratio test: 2 models fitted on 16409 observations
───────────────────────────────────────────────
     DOF  ΔDOF   Deviance  ΔDeviance  p(>Chisq)
───────────────────────────────────────────────
[1]   69        7147.5503                      
[2]   44   -25  7188.4894    40.9391     0.0233
───────────────────────────────────────────────

Results for `zcpLMM` and `zcpLMM_b` are identical. Goodness of fit is better for complex LMM -- 
marginally because 2 * ΔDOF < ΔDeviance). 

# A replication of MRK17 LMM

## Indicators

Replication of final LMM in Masson and Kliegl (2013, Table 1) as well as
reproduction of final lme4-based LMM in Masson, Rabe, and Kliegl (2017, Figure 2)

In [15]:
m3form = @formula (-1000/rt) ~ 1 + f*p*q*lq*lt +
        (1+q | Subj) + (0+lt | Subj) + (1 | Item) + (0 + p | Item) ;
mrk17_LMM = @btime fit(LinearMixedModel, m3form, dat, contrasts=contrasts);

VarCorr(mrk17_LMM)

  440.641 ms (240008 allocations: 103.89 MiB)


Variance components:
            Column      Variance      Std.Dev.    Corr.
Item     (Intercept)  0.003205660823 0.056618555
         p            0.000066922633 0.008180625   .  
Subj     (Intercept)  0.030617756083 0.174979302
         q            0.000762479409 0.027613030 -0.42
         lt           0.001062062275 0.032589297   .     .  
Residual              0.086408016139 0.293952405


Is the correlation paramter significant?

In [16]:
# remove single CP for nested LMMs
m4form = @formula (-1000/rt) ~ 1 + f*p*q*lq*lt +
        (1 | Subj) + (0+q | Subj) + (0+lt | Subj) + (1 | Item) + (0+p | Item);
rdcdLMM = @btime fit(LinearMixedModel, m4form, dat, contrasts=contrasts);

#compare nested model sequence
lrtest(rdcdLMM, mrk17_LMM)

  382.495 ms (169896 allocations: 102.11 MiB)


┌ Warning: Could not check whether models are nested as model type LinearMixedModel does not implement isnested: results may not be meaningful
└ @ Main /Users/reinholdkliegl/Google Drive/ZiF_CG_WS2/MRK17_Exp1/lrtest.jl:91


Likelihood-ratio test: 2 models fitted on 16409 observations
───────────────────────────────────────────────
     DOF  ΔDOF   Deviance  ΔDeviance  p(>Chisq)
───────────────────────────────────────────────
[1]   38        7195.8744                      
[2]   39     1  7186.8171    -9.0573     0.0026
───────────────────────────────────────────────

Yes, it is! Replicates a previous result. 

Note that `zcpLMM` and `mrk17LMM` are not nested; we cannot compare them with a LRT.

## Factors

This is an excursion with cautionary note. 
So far the replication LMM cannot be specified with factors in the RE-structure.

In [18]:
m3form_b = @formula (-1000/rt) ~ 1 + F*P*Q*lQ*lT +
        (1+Q | Subj) + zerocorr(0+lT | Subj) + zerocorr(1 + P | Item) ;
mrk17_LMM_b = @btime fit(LinearMixedModel, m3form_b, dat, contrasts=contrasts);

VarCorr(mrk17_LMM_b)

  640.107 ms (356525 allocations: 121.28 MiB)


Variance components:
            Column      Variance      Std.Dev.    Corr.
Item     (Intercept)  0.003204317379 0.056606690
         P: unr       0.000066955436 0.008182630   .  
Subj     (Intercept)  0.028859030413 0.169879458
         Q: deg       0.000762085747 0.027605901 -0.39
         lT: WD       0.003980569589 0.063091755   .     .  
         lT: NW       0.000264287779 0.016256930   .     .     .  
Residual              0.086408848803 0.293953821


This will be fixed. 

# Model comparisons

In [19]:
const mods = [cmplxLMM, zcpLMM, mrk17_LMM, rdcdLMM];
gof_summary = DataFrame(dof=dof.(mods), deviance=deviance.(mods),
              AIC = aic.(mods), AICc = aicc.(mods), BIC = bic.(mods))

,dof,deviance,AIC,AICc,BIC
,Int64,Float64,Float64,Float64,Float64
1,69,7147.55,7285.55,7286.14,7817.24
2,44,7188.49,7276.49,7276.73,7615.54
3,39,7186.82,7264.82,7265.01,7565.33
4,38,7195.87,7271.87,7272.06,7564.69


Here `dof` or degrees of freedom is the total number of parameters estimated 
in the model and `deviance` is simply negative twice the log-likelihood at 
convergence, without a correction for a saturated model.  The There information 
criteria are on a scale of "smaller is better" and all would select `mrk17_LMM` as "best".

# Illustration of crossing and nesting of factors

There is an implementation of Wilkinson & Rogers (1973) formula syntax, allowing the specification of factors not only as crossed, but also as nested in the levels of another factor or combination of factors. We illustrate this functionality with a subset of the MRK17 data. (We use oviLMM as RE structure and rt as dependent variable.)

## Crossing factors

The default analysis focuses on crossed factors yielding main effects and interactions.

In [20]:
m5form = @formula rt ~ 1 + F*P + (1 | Subj) + (1 | Item);
crossedLMM = @btime fit(LinearMixedModel, m5form, dat, contrasts=contrasts)

  18.269 ms (8610 allocations: 7.94 MiB)


Linear mixed model fit by maximum likelihood
 rt ~ 1 + F + P + F & P + (1 | Subj) + (1 | Item)
     logLik        -2 logLik          AIC             BIC       
 -1.07702642×10⁵  2.15405285×10⁵  2.15419285×10⁵  2.15473224×10⁵

Variance components:
            Column    Variance   Std.Dev.  
Item     (Intercept)    600.6089  24.507323
Subj     (Intercept)   6858.5417  82.816313
Residual              28526.1483 168.896857
 Number of obs: 16409; levels of grouping factors: 240, 73

  Fixed-effects parameters:
──────────────────────────────────────────────────────
                 Estimate  Std.Error  z value  P(>|z|)
──────────────────────────────────────────────────────
(Intercept)     647.616      9.90951    65.35   <1e-99
F: LF             7.76838    2.06014     3.77   0.0002
P: unr            7.13002    1.31934     5.40   <1e-7 
F: LF & P: unr    3.83349    1.31924     2.91   0.0037
──────────────────────────────────────────────────────

Main effects of frequency (F) and priming (P) and their interaction are significant.

In [21]:
cellmeans = by(dat, [:F, :P], 
            meanRT = :rt => mean, sdRT = :rt => std, n = :rt => length)

,F,P,meanRT,sdRT,n
,Categorical…,Categorical…,Float64,Float64,Int64
1,HF,rel,636.888,192.78,4154
2,HF,unr,642.603,184.051,4124
3,LF,rel,644.332,186.599,4107
4,LF,unr,665.376,195.107,4024


# Nesting factors

The interaction tests whether lines visualizing the interaction are parallel, 
but depending on the theoretical context one might be interested whether the
priming effect is significant high frequency targets and for low frequency targets. 
In other words, the focus is on whether the priming effect is significant for 
the levels of the frequency factor.

In [22]:
m6form = @formula rt ~ 1 + F/P + (1 | Subj) + (1 | Item);
nestedLMM = @btime fit(LinearMixedModel, m6form, dat, contrasts=contrasts)

  17.972 ms (8392 allocations: 7.55 MiB)


Linear mixed model fit by maximum likelihood
 rt ~ 1 + F + F & P + (1 | Subj) + (1 | Item)
     logLik        -2 logLik          AIC             BIC       
 -1.07702642×10⁵  2.15405285×10⁵  2.15419285×10⁵  2.15473224×10⁵

Variance components:
            Column    Variance   Std.Dev.  
Item     (Intercept)    600.6089  24.507323
Subj     (Intercept)   6858.5412  82.816310
Residual              28526.1483 168.896857
 Number of obs: 16409; levels of grouping factors: 240, 73

  Fixed-effects parameters:
──────────────────────────────────────────────────────
                 Estimate  Std.Error  z value  P(>|z|)
──────────────────────────────────────────────────────
(Intercept)     647.616      9.90951    65.35   <1e-99
F: LF             7.76838    2.06014     3.77   0.0002
F: HF & P: unr    3.29654    1.85707     1.78   0.0759
F: LF & P: unr   10.9635     1.87441     5.85   <1e-8 
──────────────────────────────────────────────────────

The results show that the priming effect is not significant for high-frequency
targets. The estimates are the differences of the cell means from the grand 
mean (i.e., 2 x estimate = effect).